In [13]:
#Install Dependancies
#%pip install tqdm
import os
import shutil
import tempfile
from tqdm import tqdm

#Clone and Install Javi's nipype library
# !git clone https://github.com/jrasero/my-nipype-pipelines.git
# !cd my-nipype-pipelines
# %pip install -U .

from my_nipype_pipelines.first_level import create_first_level_spm_wf
from nipype.interfaces.matlab import MatlabCommand

In [8]:
# Add spm12 to matlab path
MatlabCommand.set_default_paths(['C:/Users/robert/Documents/MATLAB/spm12', 
                                 'C:/Users/robert/Documents/MATLAB/spm12/toolbox'])

MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

In [9]:
#Define Contrast parameters
repetition_time = 0.75
high_pass = 128.
fwhm =  6.0
contrasts = [["Left vs Right", "T", ["Left", "Right"], [1,-1]]]

In [ ]:
for run_id in tqdm.tqdm([1,2,3]):
    # here the paths to your bold images, confounders files and event files
    bold_img = os.path.abspath(f"../BOLD_T1w/sub-811_ses-02_task-lokicat_run-0{run_id}_space-T1w_desc-preproc_bold.nii.gz")
    conf_file = os.path.abspath(f"../confounds/sub-811_ses-02_task-lokicat_run-0{run_id}_desc-confounds_regressors.tsv")
    events_file = os.path.abspath(f"../LR_events/sub-0811_ses02_task-lokicat_run0{run_id}_motor_events.tsv")
    
    # Output directory per run
    output_dir = os.path.abspath(f"../maps/sub_811_ses_02_run_0{run_id}")
    if os.path.exists(output_dir) is False:
            os.mkdir(output_dir)
            
    # Create working directory, where intermediate files from nipype are generated
    working_dir = tempfile.mkdtemp()
    wf = create_first_level_spm_wf(bold_img=bold_img, 
                                   confounders_file=conf_file, 
                                   events_file=events_file,
                                   output_dir=output_dir,
                                   repetition_time=repetition_time, 
                                   high_pass=high_pass, 
                                   fwhm=fwhm, 
                                   contrasts=contrasts)
    wf.base_dir = working_dir
    wf.run()
    
    # remove working directory
    shutil.rmtree(working_dir)
